# Классификация токсичных комментариев

Задача: обучить модель, классифицирующую токсичность комментариев с `f1 score` выше `.78`.

**Данные:**  
159292 строк с комменатриями + таргет (токсичный или нет)

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Импорты

In [1]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

import transformers

import evaluate

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import f1_score

import lightgbm as lgbm

from tqdm.auto import tqdm

In [2]:
# import warnings
# warnings.filterwarnings('ignore')

In [3]:
transformers.__version__

'4.21.3'

Сразу проверим наличие устройства с поддержкой `cuda` и поместим в переменную для дальнейшего использования.

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [5]:
seed_val = 42

np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [6]:
try:
    df_full = pd.read_csv('/datasets/toxic_comments.csv',index_col=0)
except:
    df_full = pd.read_csv('toxic_comments.csv',index_col=0)
    

In [7]:
df_full

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159449,And it looks like it was actually you who put ...,0


Видим неочищенный текст. Поскольку будем использовать bert токенизатор, то нет необходимость его преобрабатывать руками.

In [8]:
df_full['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Видим дисбаланс классов. Предпочтительный метод - оптимизация трешхолда. Но, опять же, с бертом, скорее всего это не понадобится.

# Предобработка

Разделим на тест, вал и трейн. Предварительно возьмем только 30тыс. строк и для каждой выборке по 10 тыс., чтобы ускорить обучение.

In [9]:
df = df_full.sample(30000)

In [10]:
def df_split():
    X,y = df.drop(columns=['toxic']),df['toxic']
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.6666,stratify=y)
    X_val,X_test,y_val,y_test = train_test_split(X_test,y_test,test_size=0.5,stratify=y_test)
    return X_train,X_val,X_test,y_train,y_val,y_test

X_train,X_val,X_test,y_train,y_val,y_test = df_split()

In [11]:
for x in [X_train,X_val,X_test,y_train,y_val,y_test]:
    print(x.shape)

(10002, 1)
(9999, 1)
(9999, 1)
(10002,)
(9999,)
(9999,)


# Pretrained toxic-bert

Возьмем предобученную модель `'toxic-bert'` и проверим ее на валидационной выборке для сравнения с другими моделями.  
[ссылка на описание](https://huggingface.co/unitary/toxic-bert)  

Использовать будем `pipeline` для упрощения.

Инстанциируем модель сразу на видеокарте. `device=0`

In [12]:
pretrained_toxic_bert_pipe = transformers.pipeline(model='unitary/toxic-bert',device=0)

Downloading:   0%|          | 0.00/967 [00:00<?, ?B/s]

Отправим валидационную выборку в пайплайн. Обрежем кол-во токенов до 512 шт. Применим паддинг, где необходимо. И батч сайз = 32.

In [13]:
%%time
preds = pretrained_toxic_bert_pipe(X_val['text'].tolist(),padding=True,max_length=512,truncation=True,batch_size=32)

CPU times: total: 5min 23s
Wall time: 5min 33s


In [14]:
preds[:5]

[{'label': 'toxic', 'score': 0.0008983758161775768},
 {'label': 'toxic', 'score': 0.0009383891010656953},
 {'label': 'toxic', 'score': 0.0005839156801812351},
 {'label': 'toxic', 'score': 0.0006377194658853114},
 {'label': 'toxic', 'score': 0.0006457153940573335}]

Токсик берт обучался для мультикатегориального предсказания, тип токсичности, но нам нужна только оценка токсичности в принципе, поэтому поднадобится только `score`.

In [15]:
# preds_label = []
preds_score = []
for x in preds:
    # preds_label.append(x['label'])
    preds_score.append(x['score'])


Посмотрим `f1_score` для валидационной выборки.

In [16]:
f1_score(y_val,list(map(int,np.array(preds_score) >= 0.5)))

0.9362348178137653

**Pretrained toxic bert F1 score: 93**

Это очень хороший скор для нашей задачи.

# Distillbert for Sequence Classification

Попробуем самостоятельно зафайнтюнить [дистиллберт](https://huggingface.co/docs/transformers/model_doc/distilbert). Попробуем сделать это с помощью huggingface `trainer` [ссылка](https://huggingface.co/docs/transformers/main_classes/trainer) и нативно, средствами пайторча.

## with huggingface trainer

In [ ]:
# tokenizer = transformers.AutoTokenizer.from_pretrained('distilbert-base-uncased')

# def encode_(text:pd.Series,max_length=64):
#     tokens = tokenizer(text.tolist(),
#                        add_special_tokens=True,
#                        max_length=max_length,
#                       padding=True,
#                       truncation=True,
#                       return_tensors='pt')

#     ids = tokens['input_ids']
#     mask = tokens['attention_mask']
#     return ids,mask

# train_ids,train_mask = encode_(X_train['text'])
# val_ids,val_mask = encode_(X_val['text'])



# train_dataset = TensorDataset(train_ids,train_mask,torch.tensor(y_train.values))
# val_dataset = TensorDataset(val_ids,val_mask,torch.tensor(y_val.values))

# model = transformers.AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

# metric = evaluate.load('f1')

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     # predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=logits, references=labels)

# training_args = transformers.TrainingArguments(
#     output_dir="./results",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=2,
#     weight_decay=0.01,
# )

# trainer = transformers.Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer,
#     # data_collator=data_collator,
# )

# trainer.train()

К сожалению, мне не удалось до конца разобраться с `trainer`, поэтому я предпочел классический вариант с пайторчем.

## Manual pytorch train

Очистим кэш `cuda`, чтобы не перезрузить память.

In [19]:
torch.cuda.empty_cache()

Инстанциируем токенайзер дистилберта.

In [20]:
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-uncased")

`tokenizer.encode_plus`, который используется в этой функции, возвращает и айдишники и `attention_mask`. Все это нам нужно будет подавать на вход модели для файнтюнинга.

In [21]:
def encode_(text:pd.Series,labels:pd.Series,max_length=512):
    '''
    example:
    
    encode_(df['text'],df['toxic'])
    '''
    input_ids = []
    attention_masks = []

    for t in text.values:
        encoded_dict = tokenizer.encode_plus(t,
                                            add_special_tokens=True,
                                            max_length=max_length,
                                            # pad_to_max_length=True,
                                            truncation=True,
                                            padding='max_length',
                                            return_attention_mask=True,
                                            return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    # input_ids.to(device)
    attention_masks = torch.cat(attention_masks, dim=0)
    # attention_masks.to(device)
    labels = torch.tensor(labels.values)
    # lab els.to(device)
    
    return input_ids,attention_masks,labels

input_ids,attention_masks,labels = encode_(X_train['text'],y_train,max_length=64)

Создадим пайторчевские датасеты.

In [22]:
train_dataset = TensorDataset(input_ids,attention_masks,labels)
val_dataset = TensorDataset(*encode_(X_val['text'],y_val,max_length=64))

Определим батч сайз в 32.

In [23]:
batch_size = 32

Создадим пайторчевский `DataLoader`. Один тренировочный с перемешиванием, другой валидационный без перемешивания.

In [24]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

Инстанциируем специальный класс `AutoModelForSequenceClassification`, у которого в конце находится слой для предсказания категории и укажем кол-во классов, которые нам нужно указывать (2). И сразу же отправим модель на видеокарту.

In [25]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(
                                                                    "distilbert-base-uncased", 
                                                                    num_labels = 2, 
                                                                    # output_attentions = False, 
                                                                    # output_hidden_states = False,
                                                                    )

model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

В туториалах рекомендуется использовать `AdamW` с таким лернинг рейтом. Нам этого достаточно, чтобы иметь возможность сравнивать модели.  
Также с помощью `get_scheduler` определим как будет меняться `learning_rate` с каждой итерацией.

In [26]:
optimizer = torch.optim.AdamW(model.parameters(),
                             lr = 5e-5,
                             # eps = 1e-8
                             )

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = transformers.get_scheduler(name="linear", 
                                          optimizer=optimizer, 
                                          num_warmup_steps=0, 
                                          num_training_steps=num_training_steps
                                        )

Дообучим модель, разместив все тензоры на видеокарте. И поскольку в модели мы указываем `labels`, то она возвращает и функцию потери, которую в этом случае не нужно указывать заранее.

In [27]:
%%time
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        ids = batch[0].to(device)
        mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model(ids,attention_mask=mask,labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/939 [00:00<?, ?it/s]

CPU times: total: 55.9 s
Wall time: 3min 40s


Проверим `f1_score` на валидационной выборке с помощью метрики из библиотеки `evaluate` [ссылка на документацию](https://huggingface.co/docs/evaluate/index)

In [28]:
%%time

progress_bar = tqdm(range(len(val_dataloader)))

metric = evaluate.load("f1")
model.eval()
for batch in val_dataloader:
    ids = batch[0].to(device)
    mask = batch[1].to(device)
    labels = batch[2].to(device)
    with torch.no_grad():
        outputs = model(ids,attention_mask=mask,labels=labels)

    logits = outputs.logits
    predictions = torch.ge(logits, 0.5).int()
    metric.add_batch(predictions=predictions[:,1], references=labels)
    progress_bar.update(1)

metric.compute()

  0%|          | 0/313 [00:00<?, ?it/s]

CPU times: total: 20.5 s
Wall time: 22.2 s


{'f1': 0.7816091954022989}

**Fine tuned distilbert F1 score: 78**

Скор хуже, но все равно выше целевого.

# distilbert features (embedings) to logreg, LGBM

Проверим эффективность использования эбмедингов дистилберта в логистической регресии и LGBM.

In [30]:
torch.cuda.empty_cache()

Нам не нужно перемешивание в тренировочном датасете, потому что мы извлекаем эмбединги. Поэтому они должны идти в том же порядке, что и в данных.  
Создадим загрузчики без перемешивания.

In [31]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

Используем другой класс, который поможет нам вытащить именно эмбединги.

In [32]:
model = transformers.AutoModel.from_pretrained('distilbert-base-uncased',
                                               num_labels=2)
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

При использовании полного набора данных (~130к строк), если выводить эмбединги из видеокарты на обычную оперативную память, с помощью `.cpu().numpy()` и складывать все это в список питона, то возникает ошибка недостатки памяти. Поэтому сначала соберем все эмбединги в тензор на видеокарте, затем переведем в массив нампай. Это позволит нам сэкономить оперативную память от затратных наслоений тензоров, массивов нампай и списков питона.

In [33]:
%%time
embedings_train = torch.Tensor().to(device)
for batch in train_dataloader:
    ids = batch[0].to(device)
    mask = batch[1].to(device)
    # labels = batch[2].to(device)
    with torch.no_grad():
        outputs = model(ids,attention_mask=mask)

    embedings_train = torch.cat((embedings_train,outputs[0][:,0,:]),0)

CPU times: total: 19.1 s
Wall time: 19.4 s


Проверим, что размер идентичен тренировочным данным.

In [34]:
embedings_train.size()

torch.Size([10002, 768])

Получилось 768 фичей.

Переведем в массив нампай.

In [35]:
embedings_train = embedings_train.cpu().numpy()

Снова проверим размер.

In [36]:
embedings_train.shape

(10002, 768)

Повторим для валидационной выборки.

In [37]:
%%time
embedings_val = torch.Tensor().to(device)
for batch in val_dataloader:
    ids = batch[0].to(device)
    mask = batch[1].to(device)
    # labels = batch[2].to(device)
    with torch.no_grad():
        outputs = model(ids,attention_mask=mask)

    embedings_val = torch.cat((embedings_val,outputs[0][:,0,:]),0)

CPU times: total: 19.4 s
Wall time: 19.5 s


In [38]:
embedings_val.size()

torch.Size([9999, 768])

In [39]:
embedings_val = embedings_val.cpu().numpy()

In [40]:
embedings_val.shape

(9999, 768)

In [41]:
torch.cuda.empty_cache()

## Logistic regression 

Обучим логистическую регрессию на эмбедингах.

In [42]:
logreg = LogisticRegression()

logreg.fit(embedings_train,y_train)

C:\Users\SnakeRZR\Desktop\projects\general_study\env\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Нет схождения. Посмотрим на эмбединги.

In [47]:
embedings_train[:10,0]

array([ 0.03540909, -0.03289857, -0.07736523,  0.00130919,  0.10840461,
       -0.16480303,  0.12440461,  0.1297018 , -0.10020078,  0.22269179],
      dtype=float32)

Они выглядят нормированными, поэтому здесь, возможно, дело в кол-ве фичей. Можно увеличить кол-во итераций или изменить solver.

In [48]:
pred = logreg.predict(embedings_val)
f1_score(y_val,pred)

0.6754436176302232

Скор неудовлетворительный.  
Попробуем оптимизировать трешхолд.

In [49]:
def optimize_threshold(predict_proba,y_true,score_func,labels,**kwargs):
    best_threshold = 0
    best_score = 0
    
    for i in range(1,100):
        threshold = i / 100
        prediction_with_threshold = list(map(int, predict_proba >= threshold))
        score = score_func(y_true,prediction_with_threshold, **kwargs)
        if score > best_score:
            best_score = score
            best_threshold = threshold
            
    # display_confusion_matrix(y_true,list(map(int,predict_proba >= best_threshold)),labels=labels)
    return best_threshold, best_score

In [50]:
predict_proba = logreg.predict_proba(embedings_val)[:,1]

best_threshold, best_score = optimize_threshold(predict_proba,
                                                y_val,
                                               f1_score,
                                                # recall_score,
                                               logreg.classes_,
                                               zero_division=0)
print(best_threshold, best_score)
            

0.32 0.7028372324539572


**Logreg F1 score: 70**

Получили 3 пункта. Скор все еще неудовлетворительный.

## LGBM

Проверим LGBM на эмбедингах.

Здесь не указываем функцию метрики, на которую бы ориентировался алгоритм. Нам достаточно метрики поумолчанию, чтобы сравнить с другими моделями.

In [51]:
model = lgbm.LGBMClassifier()
model.fit(embedings_train,y_train,eval_set=(embedings_val,y_val)
         )

[1]	valid_0's binary_logloss: 0.293322
[2]	valid_0's binary_logloss: 0.275291
[3]	valid_0's binary_logloss: 0.26142
[4]	valid_0's binary_logloss: 0.251155
[5]	valid_0's binary_logloss: 0.242056
[6]	valid_0's binary_logloss: 0.235126
[7]	valid_0's binary_logloss: 0.228391
[8]	valid_0's binary_logloss: 0.222526
[9]	valid_0's binary_logloss: 0.217147
[10]	valid_0's binary_logloss: 0.212858
[11]	valid_0's binary_logloss: 0.209222
[12]	valid_0's binary_logloss: 0.205744
[13]	valid_0's binary_logloss: 0.202059
[14]	valid_0's binary_logloss: 0.19944
[15]	valid_0's binary_logloss: 0.196728
[16]	valid_0's binary_logloss: 0.194417
[17]	valid_0's binary_logloss: 0.192166
[18]	valid_0's binary_logloss: 0.190682
[19]	valid_0's binary_logloss: 0.18895
[20]	valid_0's binary_logloss: 0.187132
[21]	valid_0's binary_logloss: 0.185502
[22]	valid_0's binary_logloss: 0.184079
[23]	valid_0's binary_logloss: 0.182902
[24]	valid_0's binary_logloss: 0.181835
[25]	valid_0's binary_logloss: 0.180674
[26]	valid_0

LGBMClassifier()

In [52]:
pred = model.predict(embedings_val)
f1_score(y_val,pred)

0.6357615894039735

In [53]:
predict_proba = model.predict_proba(embedings_val)[:,1]

best_threshold, best_score = optimize_threshold(predict_proba,
                                                y_val,
                                               f1_score,
                                                # recall_score,
                                               logreg.classes_,
                                               zero_division=0)
print(best_threshold, best_score)

0.17 0.6683192860684184


**LGBM F1 score: 66**

Скор неудовлетворительный.

**Лучшая модель**: `toxic-bert`  
Проверим ее на тестовых данных.

# Best model test evaluation

Окончательно проверим модель на тестовых данных.

In [54]:
%%time
preds = pretrained_toxic_bert_pipe(X_test['text'].tolist(),padding=True,max_length=512,truncation=True,batch_size=32)

preds_score = []
for x in preds:
    # preds_label.append(x['label'])
    preds_score.append(x['score'])

f1_score(y_test,list(map(int,np.array(preds_score) >= 0.5)))

CPU times: total: 5min 24s
Wall time: 5min 32s


0.9488607594936709

Скор стабильно высокий.

# Выводы
* Лучше всего себя показала предобученная модель `toxic-bert` (`f1_score` на валидационных = `0.93`, на тестовых = `0.94`)
* На втором месте distilbert дообученный на настоящих данных (`f1_score = 0.78`)
* Логистическая регрессия и lgbm обученные на эмбедингах distilbert показали `70` и `66`.
